### Pipeline optimization 
* Objective:
 * Build a stack of numpy arrays depicting the cost distance away from biogas sources
 
* Requirements - Packages
 * NumPy, Pandas, Scikit-Image

* Requirements - Data:
 * CSV of biogas sources, including coordinates and biogas production potential (MMBtu)
 * Cost surface for building pipelines
 
* Output: 
 * Stacked NumPy array

In [1]:
#Import packages
import numpy as np
import pandas as pd
from skimage import graph
from osgeo import gdal, osr
import geopandas as gpd
from shapely.geometry import Point

In [2]:
#Read in biogas sources (example: Duplin Co)
dfBG =  pd.read_excel('../data/DuplinCountySwineFarmEconomics.xlsx',
                         sheet_name='Duplin County Swine Farm Master').iloc[:,[11,12,-6,-1]]
dfBG.sort_values(by='Total Potential Methane Yield (scf/h)',ascending=False,inplace=True)

In [3]:
import os, sys
env_folder = os.path.dirname(sys.executable)
print(os.path.join(env_folder,'Library','share'))
os.environ['PROJ_LIB']=os.path.join(env_folder,'Library','share')

C:\Users\jpfay\AppData\Local\ESRI\conda\envs\bg_optimi\Library\share


In [4]:
geom = [Point(xy) for xy in zip(dfBG.iloc[:,1],dfBG.iloc[:,0])]
gdfBG = gpd.GeoDataFrame(dfBG, geometry = geom)
gdfBG.crs = {'init':'epsg:4326'}

In [5]:
#Read in cost surface
#ds = gdal.Open('../data/MIT Cost Surface Duplin Co/MIT Cost Surface Duplin.tif')
ds =  gdal.Open('../scratch/MIT_CostSurface_500m.img')
#Get the raster projection
ds_prj = ds.GetProjection()
#Get raster attributes
llx, x_size, x_angle, lly, y_angle, y_size = ds.GetGeoTransform()
#Extract Band1 as the cost array
arrCost = np.array(ds.GetRasterBand(1).ReadAsArray())
#Set zero costs to high costs
arrCost[arrCost == 0] = 10000
#Print info
print("Lower left coordinate = ({0:.8f},{1:.8f})".format(llx,lly))
print("Pixel size is {0:.8f}(x), {1:.8f}(y)".format(x_size,y_size))

Lower left coordinate = (1502530.32985527,-361423.20573852)
Pixel size is 500.00000000(x), -500.00000000(y)


In [6]:
#Project the points to match the raster
gdf2 = gdfBG.to_crs(ds_prj)

In [7]:
gdf2.iloc[0,3]

0.019107790334014153

In [8]:
#Create the MCP object from the cost surface
print("Creating the graph from the cost raster")
lc_graph = graph.MCP_Geometric(arrCost,sampling=(x_size,y_size))

Creating the graph from the cost raster


In [9]:
#Iterate through all records and create a cost distance raster
cd_arrays = []
dx = {}
for i,r in gdf2.iterrows():
    print(".",end='')
    x = r.geometry.x
    y = r.geometry.y
    z = r[3]
    
    #Get array index from lat long
    xOffset = int(round((x - llx)/x_size))
    yOffset = int(round((y - lly)/y_size))

    #Compute cost distances away
    lc_graph = graph.MCP_Geometric(arrCost * z,sampling=(x_size,y_size))
    cd_array = lc_graph.find_costs(starts=([(yOffset, xOffset)]))[0]
    cd_arrays.append(cd_array)
    dx[i]=cd_array

................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [10]:
#Stack all the arrays
arrStack =np.stack(cd_arrays)
np.save('../data/DuplinStack500m.npy',arrStack)

In [11]:
#Reduce to minimum costs
arrMin = np.amin(arrStack,axis=0)

In [ ]:
#Export to geotiff
bands,height,width = arrStack.shape
drv = gdal.GetDriverByName("GTiff")
dsOut = drv.Create('../scratch/min_cost.tif',width,height,1,gdal.GDT_Float32)

dsOut.GetRasterBand(1).WriteArray(arrMin)
dsOut.SetGeoTransform (ds.GetGeoTransform())
dsOut.SetProjection(ds_prj)
dsOut.FlushCache()

In [13]:
#Export to geotiff
bands,height,width = arrStack.shape
drv = gdal.GetDriverByName("GTiff")
dsOut = drv.Create('../scratch/site1.tif',width,height,1,gdal.GDT_Float32)

dsOut.GetRasterBand(1).WriteArray(arrStack[0])
dsOut.SetGeoTransform (ds.GetGeoTransform())
dsOut.SetProjection(ds_prj)
dsOut.FlushCache()